# Tasks

Tasks are used to build up experiment and analysis workflows.
The library provides generic tasks for building, compiling and running LabOne Q experiments.
It also provides specific tasks for simple experiments and the associated analysis (e.g. Rabi).

Let's get started.

## Setting up a device and session

Build your LabOne Q `DeviceSetup`, qubits and `Session` as normal. Here we import an example from the applications library's test suite (this will change in the near future):

In [ ]:
from laboneq.simple import *

from laboneq_applications.qpu_types.tunable_transmon import demo_platform

In [ ]:
# Create a demonstration QuantumPlatform for a tunable-transmon QPU:
qt_platform = demo_platform(n_qubits=6)

# The platform contains a setup, which is an ordinary LabOne Q DeviceSetup:
setup = qt_platform.setup

# And a tunable-transmon QPU:
qpu = qt_platform.qpu

# Inside the QPU, we have qubits, which is a list of six LabOne Q Application
# Library TunableTransmonQubit qubits:
qubits = qpu.qubits

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)

## Tasks

When running experiments with LabOne Q there are usually a few higher-level steps that one has to perform:

- Build a DSL experiment.
- Compile the experiment.
- Run the experiment.
- Analyze the experiment.

In the applications library, we call these steps *tasks*. The library provides some predefined tasks and you can also write your own.

Let's use some of the predefined tasks to run an amplitude Rabi experiment.

### Using provided tasks to run an experiment

In [ ]:
import numpy as np
from laboneq import workflow
from laboneq.simple import dsl
from laboneq.workflow.tasks import compile_experiment, run_experiment

from laboneq_applications.experiments.amplitude_rabi import create_experiment
from laboneq_applications.experiments.options import TuneupExperimentOptions
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonOperations,
)

In [ ]:
create_experiment.src

Let's create, compile and run the rabi experiment with some simple input parameters.

In [ ]:
qop = TunableTransmonOperations()
amplitudes = np.linspace(0.0, 1.0, 10)
options = TuneupExperimentOptions(count=10)
exp = create_experiment(qpu, [qubits[0]], [amplitudes], options)
compiled_exp = compile_experiment(session, exp)
result = run_experiment(session, compiled_exp)

And let's examine the rabi measurement results:

In [ ]:
result.q0.result

And the measurements of the 0 and 1 states for calibration:

In [ ]:
result.q0.cal_trace.g

In [ ]:
result.q0.cal_trace.e

Each of `create_experiment`, `compile_experiment` and `run_experiment` is a task. They are ordinary Python functions, but they provide some special hooks so that they can be incorporate into workflows later.

Like quantum operations, they can be inspected. Let's inspect the source code of the `create_experiment` task of the rabi to see how the rabi experiment is created. You can inspect the source code of any task.

In [ ]:
# docstring
create_experiment?

In [ ]:
# source code
create_experiment.src

### Writing your own tasks

As mentioned, tasks are mostly just ordinary Python functions. You can write your own task as follows:

In [ ]:
@workflow.task
def add(a, b):
    return a + b

In [ ]:
add(1, 2)

### Writing an experiment task

Built-in tasks like `compile_experiment` and `run_experiment` are quite standard and you shouldn't need to write your own versions very often, but tasks that build experiments, such as `create_experiment` for the rabi, will often be written by you.

Let's write our own version of the `create_experiment` for the rabi experiment that sweeps pulse lengths instead of amplitudes.

In [ ]:
from laboneq.dsl.parameter import SweepParameter

In [ ]:
@workflow.task
@dsl.qubit_experiment
def duration_rabi(
    qop,
    q,
    q_durations,
    count,
    transition="ge",
    cal_traces=True,
):
    """Pulse duration Rabi experiment."""
    with dsl.acquire_loop_rt(
        count=count,
    ):
        with dsl.sweep(
            name=f"durations_{q.uid}",
            parameter=SweepParameter(f"durations_{q.uid}", q_durations),
        ) as length:
            qop.prepare_state(q, transition[0])
            qop.x180(q, length=length, transition=transition)
            qop.measure(q, f"{q.uid}/result")
            qop.passive_reset(q)

        if cal_traces:
            with dsl.section(
                name=f"cal_states/{q.uid}",
            ):
                for state in transition:
                    qop.prepare_state(q, state)
                    qop.measure(q, f"{q.uid}/cal_trace/{state}")
                    qop.passive_reset(q)

Quite a few new constructions and quantum operations have been introduced here, so let's take a closer look:

* `transition="ge"`: Each kind of qubit supports different transitions. For the tunable transmon qubits implement in the applications library the two transitions are `"ge"` (i.e. ground to first excited state) and `"ef"` (i.e. first to second excited state). The tunable transmon operations accept the transition to work with as a parameter.

* `with dsl.sweep`: This is an ordinary DSL sweep.

* `qop.prepare_state`: Prepare the specified qubit state. The tunable transmon `prepare_state` accepts `"g"`, `"e"` and `"f"` as states to prepare.

* `qop.passive_reset`: This operation resets the qubit to the ground state by delaying for an amount of time configured in the calibration.

* `if cal_traces:`: An ordinary Python `if` statement. It allows the calibration traces to be omitted if requested by the parameters.

* `dsl.section`: This creates a new section in the experiment. Sections are important to create timing-consistent and reproducible behavior.

In [ ]:
qop = TunableTransmonOperations()
durations = np.linspace(10.0e-9, 50e-9, 10)
count = 10

exp = duration_rabi(qop, qubits[0], durations, count=count)
compiled_exp = compile_experiment(session, exp)
result = run_experiment(session, compiled_exp)

In [ ]:
result.q0.result

In [ ]:
result.q0["cal_trace"]["g"]

In [ ]:
result.q0["cal_trace"]["e"]

### Why have tasks at all?

So far what we've seen of tasks don't provide much beyond encouraging some structure. Encouraging structure is valuable, but the motivation behind tasks is what's to come in the following tutorials:

* Being able to produce a well-organised experiment record when tasks are used in workflows.
* Being able to supply global options to tasks in a structured way.
* Being able to recover partial results when errors occur.
* Being able to pause and resume workflow execution.
* Being able to build complex dynamic workflows that can execute tasks conditionally and dynamically add tasks.